In [ ]:
# Install necessary libraries
!pip install transformers
!pip install datasets
!pip install torch torchvision torchaudio  # PyTorch
!pip install nltk
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install tqdm


In [ ]:
# # Step 1: Install necessary libraries
# !pip install transformers datasets torch torchvision torchaudio pandas scikit-learn tqdm

# # Step 2: Import libraries
# import pandas as pd
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset
# # Step 1: Install necessary libraries
# !pip install transformers datasets torch torchvision torchaudio pandas scikit-learn tqdm

# # Step 2: Import libraries
# import pandas as pd
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset

# Step 3: Load the Sentiment140 dataset
dataset = load_dataset('sentiment140')

# Step 4: Preprocess the dataset
# Rename the 'sentiment' column to 'target'
# The dataset has 'text' and 'sentiment' as the main columns
dataset = dataset.rename_column("sentiment", "target")

# Step 5: Map the sentiment values (0 for negative, 4 for positive) to 0 and 1
dataset = dataset.map(lambda x: {'target': 0 if x['target'] == 0 else 1})

# Step 6: Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Step 7: Set format for PyTorch
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'target'])

# Step 8: Split the dataset into train and test
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

# Step 9: Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 10: Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Step 11: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Step 12: Fine-tune the model
trainer.train()

# Step 13: Evaluate the model
trainer.evaluate()

# Step 14: Save the model
model.save_pretrained('./sentiment_model')
tokenizer.save_pretrained('./sentiment_model')


Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: